# Отчет по заданию

## Описание задачи
В базе данных информация о клиентах, оформленных на сайте заказах и маркетинговых коммуникациях хранится в нескольких таблицах (см. описание ниже). Предположим, что в базе содержатся все доступные данные по компании.<br/>
Для примера данные таблиц выгружены в файл «Тестовое задание_market_CRM» и приведены на разных листах.

## Описание данных
***Таблица Orders – данные о заказах:***
1.	OrderID – идентификатор заказа, целое типа int.
2.	CustomerID – идентификатор клиента, целое типа int.
3.	OrderDate – дата оформления заказа, тип nchar(8), формат записи - "YYYYMMDD".
4.	OrderState – состояние заказа, строка типа nvarchar(9).
Возможные значения: "Fulfilled" (исполнен) и "Cancelled" (отменен).
5.	DeliveryDays – количество дней от оформления заказа до получения его клиентом, целое типа tinyint. 
6.	Utmkey – идентификатор канала, из которого перешёл клиент, целое типа tinyint.

Первичный ключ таблицы – поле OrderID.

***Таблица Order_List – состав заказов:***
1.	OrderID – идентификатор заказа, целое типа int.
2.	SKU – идентификатор товара, целое типа int.
3.	Quantity – количество заказанного товара, целое типа tinyint.
4.	Price – стоимость одной единицы товара до применения скидки, целое типа int.
5.	Discount – относительный размер скидки, с плавающей точкой float

Первичный ключ таблицы – комбинация полей OrderID и SKU.

***Таблица Sent_Emails – отправленные коммуникации:***
1.	CustomerID – идентификатор клиента, целое типа int.
2.	Utmkey – идентификатор канала коммуникации, целое типа tinyint.
3.	Status – статус коммуникации, строка типа nvarchar
4.	action_id – идентификатор коммуникации
5.	action_type – канал распространения коммуникации, строка типа nvarchar
6.	action_name – название коммуникации, строка типа nvarchar
7.	timestamp – дата статуса коммуникации 

Первичный ключ таблицы – комбинация полей CustomerID, Utmkey, Status, timestamp

## Загружаю данные в СУБД:

Спасибо за интересную задачу!<br/>
Итак, у меня под боком есть сервер с ClickHouse. Буду делать на нем.<br/>

### Создаю базу, таблицы и загружаю в них полученные данные:

``` mysql

CREATE DATABASE IF NOT EXISTS market;

CREATE TABLE market.orders
(
OrderID Int64,
CustomerID	Int64,
OrderDate Date,
OrderState String,
DeliveryDays Nullable(Int64),
Utmkey Int64
) ENGINE = MergeTree ORDER BY (OrderID) SETTINGS index_granularity = 8192;

cat /home/dmitry/datasets/Orders.csv |clickhouse-client -h 10.8.10.119 --port 9000 -u default  -q 'INSERT INTO market.orders FORMAT CSV'


CREATE TABLE market.order_list
(
OrderID Int64,
SKU	Int64,
Quantity Int64,
Price Int64,
Discount Float64
) ENGINE = MergeTree ORDER BY (OrderID) SETTINGS index_granularity = 8192;

cat /home/dmitry/datasets/Order_list.csv |clickhouse-client -h 10.8.10.119 --port 9000 -u default  -q 'INSERT INTO market.order_list FORMAT CSV'


CREATE TABLE market.sent_emails
(
CustomerID Int64,
Utmkey	Int64,
Status String,
action_id Int64,
action_type String,
action_name String,
timestamp Date    
) ENGINE = MergeTree ORDER BY (CustomerID) SETTINGS index_granularity = 8192;

cat /home/dmitry/datasets/Sent_Emails.csv |clickhouse-client -h 10.8.10.119 --port 9000 -u default  -q 'INSERT INTO market.sent_emails FORMAT CSV'
```

## Задание №1: 
<br/>
Необходимо написать SQL-запрос (стандарт ANSI или любой диалект на выбор), который считает ретеншн рейт когорт в следующий месяц и во второй месяц после совершения первого заказа. Например, в случае январской когорты считаем выживаемость в феврале (первый месяц) и в марте (второй месяц). <br/>
Когорта определяется по месяцу совершения первого исполненного (Fulfilled) заказа. <br/>
<br/>
Результаты вывести в виде:<br/>

когорта | размер когорты | Retention rate в первый месяц | Retention rate во второй месяц
:---------|:-----------------|:---------|:----------
201801 | * | * | *
201802 | * | * | *
201803 | * | * | *

## Решение Задания №1:
<br/>
Вот такой получился код SQL запроса:
<br/>

``` mysql
select
    trim(TRAILING '-' from trim(TRAILING '01' from toString(birthmonth_))) as cohort,
    count(CustomerID) as cohort_size,
    round(sum(month_distance_1_)/count(month_distance_1_)*100,2) as retention_rate_1_month,
    round(sum(month_distance_2_)/count(month_distance_2_)*100,2) as retention_rate_2_month
from
    (
    select
        any(birthmonth) as birthmonth_,
        CustomerID,
        max(month_distance_1) as month_distance_1_,
        max(month_distance_2) as month_distance_2_
    from
        (
        select 
            l.OrderID,
            l.CustomerID,
            r.birthmonth,
            toStartOfMonth(l.OrderDate) as ordermonth,
            toMonth(l.OrderDate) - toMonth(r.birthmonth) as month_distance,
            CASE
            WHEN toMonth(l.OrderDate) - toMonth(r.birthmonth) == 1 THEN 1
            ELSE 0
            END as month_distance_1,
            CASE
            WHEN toMonth(l.OrderDate) - toMonth(r.birthmonth) == 2 THEN 1
            ELSE 0
            END as month_distance_2
        from 
            market.orders as l
        left join
            (
            select
                CustomerID,
                min(toStartOfMonth(OrderDate)) as birthmonth
            from
                orders
            group by CustomerID
            ) as r
        on l.CustomerID = r.CustomerID
        )
    group by
        CustomerID
    )
group by
    birthmonth_
```
<br/>
Результирующая таблица на скрине внизу:
<br/>

![01.jpg](pics/01.jpg)

## Задание №2: 
<br/>
Необходимо написать SQL-запрос (стандарт ANSI или любой диалект на выбор), который считает средние затраты на удержание и привлечение в рублях по месяцам. Затраты на удержание считаем, как средний размер скидки (Discount) по заказам повторных клиентов, затраты на привлечение – средний размер скидки в заказах новых клиентов. <br/>
<br/>
Результаты вывести в виде:

месяц | затраты на привлечение | затраты на удержание
:---------|:-----------------|:---------
201801 | * | * 
201802 | * | * 
201803 | * | * 

## Решение Задания №2:
<br/>
Здесь у меня к бизнесу 2 уточняющих вопроса:<br/>
<br/>
1. В первом задании речь шла о когортах. Это задание я понял что речь идет не про когорты, а про фактические месяцы, когда возникли затраты. И к когортам их привязывать не нужно. Если все таки нужен когортный анализ - сообщите мне, я переделаю скрипт. <br/>
<br/>
2. В таблице "order_list" есть столбец "Price"  Я написал скрипт, предполагая что это сумма, которую в итоге оплатил клиент. То есть, за вычетом скидки. Если это не так, дайте мне знать, нужно будет изменить скрипт. 
<br/>
<br/>
<br/>
Вот такой получился код SQL запроса:
<br/>

``` mysql
select
    ordermonth,
    round(sum(attraction_cost)/sum(attraction_counter), 2) as attraction_cost_avg,
    round(sum(retention_cost)/sum(retention_counter), 2) as retention_cost_avg
from
    (
    select
        any(ordermonth) as ordermonth,
        OrderID,
        sum(attraction_cost) as attraction_cost,
        any(attraction_counter) as attraction_counter,
        sum(retention_cost) as retention_cost,
        any(retention_counter) as retention_counter
    from
        (
        select
            r1.ordermonth,
            l1.OrderID,
            CASE
            WHEN r1.days_distance == 0 THEN (l1.Quantity * l1.Price) / (100 - l1.Discount) * l1.Discount
            ELSE 0
            END as attraction_cost,
            CASE
            WHEN r1.days_distance == 0 THEN 1
            ELSE 0
            END as attraction_counter,
            CASE
            WHEN r1.days_distance > 0 THEN (l1.Quantity * l1.Price) / (100 - l1.Discount) * l1.Discount
            ELSE 0
            END as retention_cost,
            CASE
            WHEN r1.days_distance > 0 THEN 1
            ELSE 0
            END as retention_counter
        from
            market.order_list as l1
        inner join
            (
            select 
                l.OrderID,
                l.CustomerID,
                r.birthday,
                l.OrderDate,
                trim(TRAILING '-' from trim(TRAILING '01' from toString(toStartOfMonth(l.OrderDate)))) as ordermonth,
                l.OrderDate - r.birthday as days_distance
            from 
                market.orders as l
            left join
                (
                select
                    CustomerID,
                    min(OrderDate) as birthday
                from
                    market.orders
                group by CustomerID
                ) as r
            on l.CustomerID = r.CustomerID
            having OrderState != 'Cancelled'
            ) as r1
        on l1.OrderID == r1.OrderID
        )
    group by
        OrderID
    )
group by
        ordermonth
order by ordermonth
```
<br/>
Результирующая таблица на скрине внизу:
<br/>

![02.jpg](pics/02.jpg)

## Задание №3: 
<br/>
Посчитать показатели эффективности email рассылок: вывести размеры сегментов, click rate, open rate, количество заказов, которые сделали, перейдя по этой рассылке, суммарную стоимость заказов, средний чек. <br/>
<br/>
Результаты вывести в виде:

акция | сегмент | click rate | open rate | Заказы | Стоимость | Средний чек
:---------|:-----------------|:---------|:---------|:---------|:---------|:---------
Уже оценили «Машину кэшбэка»? | * | * | * | * | * | *
Не машина, а пушка! | * | * | * | * | * | *
Запустите Машину Кэшбэка | * | * | * | * | * | *

## Решение Задания №3:
<br/>

Записки на полях:<br/>
<br/>
Для CustomerId 4441 нет подтверждения доставки письма. Зато есть открытие и клик  Хочется спросить как принято в компании учитывать такие случаи.   
<br/>
Задача интересна тем, что есть о чем поспрашивать заказчика. <br/>
Те, кто работают в компании, наверное, точно знают, а мне интересно: Заказом, сделанным по рекламной ссылке считается только оформленный в этот же день? А если клиент набирал корзину пару дней, а на второй день сделал заказ? или ждал ЗП и сделал заказ через несколько дней после клика по рекламному письму?<br/>
Раз заказчика поспрашивать нет возможности, тогда буду писать запрос заказов, сделанных день в день с кликом.<br/> Если есть подробности по этому вопросу, то сообщите мне об этом, я переделаю скрипт SQL запроса.<br/>
Еще вопрос, а может быть несколько заказов по одному клику? Думаю, что да. Присланные данные позволяют сматчить клик и заказ только по дате. И я, как покупатель, часто делаю несколько заказов в день. Добавлю еще одну группировку на этот случай. <br/>
<br/>
Вот такой получился код SQL запроса:
<br/>

``` mysql
select
    action_id,
    any(action_name) as action_name,
    uniqExact(CustomerID) as customers_cnt,
    round(sum(click_cnt_)/sum(delivered_cnt)*100, 2) as click_rate,
    round(sum(open_cnt)/sum(delivered_cnt)*100, 2) as open_rate,
    sum(orders_cnt) as orders_count,
    CASE
    WHEN sum(revenue_) > 0 THEN sum(revenue_)
    ELSE 0
    END as revenue,
    CASE
    WHEN (sum(revenue_)/sum(orders_cnt)) > 0 THEN (sum(revenue_)/sum(orders_cnt))
    ELSE 0
    END as AOV
from
    (
    select
        any(action_id) as action_id,
        any(action_name) as action_name,
        CustomerID,
        any(delivered_cnt) as delivered_cnt,
        any(click_cnt) as click_cnt_,
        any(open_cnt) as open_cnt,
        sum(order_counter) as orders_cnt,
        sum(order_price) as revenue_
    from
        (
        select
            any(timestamp) as timestamp,
            any(Utmkey) as utmkey,
            any(action_id) as action_id,
            any(action_name) as action_name,
            CustomerID,
            sum(delivered) as delivered_cnt,
            sum(click) as click_cnt,
            sum(open) as open_cnt
        from
            (
            select 
                timestamp,
                Utmkey,
                action_id,
                action_name,
                CustomerID,
                CASE
                WHEN Status == 'delivered' THEN 1
                ELSE 0
                END as delivered,
                CASE
                WHEN Status == 'clicked' THEN 1
                ELSE 0
                END as click,
                CASE
                WHEN Status == 'opened' THEN 1
                ELSE 0
                END as open
            from 
                market.sent_emails as l
            where Status == 'delivered' or Status == 'clicked' or Status == 'opened'
            )
        group by CustomerID
        ) as l1
        
        left join
        
        (
        select
            any(OrderDate) as OrderDate,
            OrderID,
            any(CustomerID) as CustomerID,
            any(Utmkey) as Utmkey,
            sum(total_price) as order_price,
            1 as order_counter
        from
            (
            select 
                Quantity * Price as total_price,
                *
            from 
                market.order_list as l2
            inner join
                market.orders as r2
            on l2.OrderID = r2.OrderID
            where r2.OrderState != 'Cancelled'
            )
        group by OrderID
        ) as r1
        on l1.timestamp == r1.OrderDate and
            l1.CustomerID == r1.CustomerID and
            click_cnt == 1
    group by
        CustomerID
    )
group by
    action_id
order by
    action_id DESC
```
<br/>
Результирующая таблица на скрине внизу:
<br/>

![03.jpg](pics/03.jpg)